In [7]:
# ! /Users/kshitijahande/opt/anaconda3/envs/pytorch/bin/python -m pip install sklearn

     |████████████████████████████████| 7.2 MB 4.4 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=1c1f73c3a93febbf1ff281ba7209dd44f42e9c70ecddfa78b7358fd3ea02ba94
  Stored in directory: /Users/kshitijahande/Library/Caches/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


In [31]:
# ******************************************************************
# 
# Contains train and test modules with multitask loss function
# 
# Use "get_result_on_evaldata" to generate predictions on the 
# evaluation data for challenge leaderboard
# 
# ******************************************************************

import torch
import torch.nn.functional as F
import sys
from icecream import ic 


# from utils import *
from ipynb.fs.full.utils import *

from sklearn.metrics import roc_auc_score, mean_squared_error, f1_score, accuracy_score
 
# mse_criterion = torch.nn.MSELoss()
ce_criterion  = torch.nn.CrossEntropyLoss()
print(ce_criterion)

def concat_output(output, device):
    temp = torch.tensor([]).to(device)

    if output.shape[1] == 2:
        temp = torch.argmax(output[:, 0:2], dim=1).view(-1,1)
        return temp

    temp = torch.argmax(output[:, 0:2], dim=1).view(-1,1)
    temp = torch.cat((temp, torch.argmax(output[:, 2:4], dim= 1).view(-1,1)), dim= 1)
    temp = torch.cat((temp, output[:, 4].view(-1,1)), dim= 1)
    temp = torch.cat((temp, output[:, 5].view(-1,1)), dim= 1)
    return temp

def loss_func(output, target, device):

    humor_id = (target[:, 0]==1)
    target   = target.float()
    print(output[:, 0:2])
    loss1 = ce_criterion(output[:, 0:2], target[:, 0].long())	                	# is_humor binary classification
#     loss2 = ce_criterion(output[humor_id][:, 2:4], target[humor_id][:, 1].long())	# humor_controversy binary classification
#     loss3 = mse_criterion(output[humor_id][:, 4], target[humor_id][:, 2])			# humor rating regression loss
#     loss4 = mse_criterion(output[:, 5], target[:, 3])								# offensive rating regression loss
#     loss  = loss1+loss2+loss3+loss4
    loss=loss1
    # this block of generating temp is verified throughly
    temp = concat_output(output, device)
    temp = temp.detach()

    return loss, temp

def train(model, dataloader, optimizer, device):
#     ic('1')
    model.train()
#     ic('2')
    train_loss = AverageMeter()

    for batch_id, batch_data in enumerate(dataloader):
        data       = batch_data[0]
        target     = batch_data[1].to(device)

        input_id   = data[0].to(device).squeeze()
        mask_id    = data[1].to(device).squeeze()
        segment_id = data[2].to(device).squeeze()

        output = model(input_id, mask_id, segment_id)
        print(output, target, device)
        loss, _ = loss_func(output, target, device= device)
        print('outside loc_func')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.update(loss.item(), target.shape[0])

    return train_loss.avg


CrossEntropyLoss()
